In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [319]:
df = pd.read_csv("../EngagementByDevice/data/offer_group_by_device_UK.txt", sep = "\t", header=None)
df.columns = ['count_asins','device_class','offer_group','customer_id']

In [320]:
print(df.shape)
df.head()

(3512147, 4)


,count_asins,device_class,offer_group,customer_id
0,255,Living Room Devices,PRIME,A1YIYEIGGH1ZL3
1,211,Living Room Devices,PRIME,A2BWZY0WXA4NJX
2,29,Living Room Devices,PRIME,A1ZRKO6XRECAWQ
3,218,Living Room Devices,PRIME,A3IYCKI18OZWPJ
4,1,Others,PRIME,A2HH38F6X2K802


In [321]:
df['customer_id'].nunique()

1646810

In [322]:
#Dont forget to pass a list to isin!!!
df['new_groups'] = np.where( df['offer_group'].isin(["RENTAL","PURCHASE"]) == True , 'TVOD','SVOD')

In [323]:
df.head()

,count_asins,device_class,offer_group,customer_id,new_groups
0,255,Living Room Devices,PRIME,A1YIYEIGGH1ZL3,SVOD
1,211,Living Room Devices,PRIME,A2BWZY0WXA4NJX,SVOD
2,29,Living Room Devices,PRIME,A1ZRKO6XRECAWQ,SVOD
3,218,Living Room Devices,PRIME,A3IYCKI18OZWPJ,SVOD
4,1,Others,PRIME,A2HH38F6X2K802,SVOD


# Make a pivot
## reclassify into svod and tvod streamers

In [324]:
r = pd.DataFrame(df.groupby(['customer_id',"new_groups"])['count_asins'].sum().reset_index())

In [325]:
r_pivot = pd.DataFrame(pd.pivot_table(r, index = 'customer_id',columns=['new_groups'], values = 'count_asins').reset_index())

In [326]:
r_pivot = r_pivot.fillna(0)
r_pivot.head()

new_groups,customer_id,SVOD,TVOD
0,A000023039I6L41DP16K3,0.0,3.0
1,A00005183VODPLUOFIKF8,696.0,0.0
2,A00014061UOTNRBL8FLXR,10.0,0.0
3,A00024081C9YYWHJYQH4L,32.0,0.0
4,A0002574KSGGCRIX1576,200.0,0.0


In [327]:
#Create classes for bucketing customers
r_pivot['class'] = 'NA'
r_pivot['class'] = np.where( (r_pivot['SVOD'] > 0) & (r_pivot['TVOD'] == 0), "SVOD Only", r_pivot['class'])
r_pivot['class'] = np.where( (r_pivot['SVOD'] == 0) & (r_pivot['TVOD'] > 0), "TVOD Only", r_pivot['class'])
r_pivot['class'] = np.where( (r_pivot['SVOD'] > 0) & (r_pivot['TVOD'] > 0), "Hybrid", r_pivot['class'])

In [328]:
print(r_pivot.shape)
r_pivot.head()

(1646810, 4)


new_groups,customer_id,SVOD,TVOD,class
0,A000023039I6L41DP16K3,0.0,3.0,TVOD Only
1,A00005183VODPLUOFIKF8,696.0,0.0,SVOD Only
2,A00014061UOTNRBL8FLXR,10.0,0.0,SVOD Only
3,A00024081C9YYWHJYQH4L,32.0,0.0,SVOD Only
4,A0002574KSGGCRIX1576,200.0,0.0,SVOD Only


In [329]:
r_pivot.groupby("class").size()

class
Hybrid       546544
SVOD Only    991345
TVOD Only    108921
dtype: int64

In [330]:
t = df.pivot_table(index="customer_id",columns = ['new_groups','offer_group','device_class'], values = 'count_asins')
t = pd.DataFrame(t.reset_index())
t = t.fillna(0)

In [331]:
t.head()

new_groups              customer_id                SVOD                        \
offer_group                                       PRIME                         
device_class                        Living Room Devices Mobile Devices Others   
0             A000023039I6L41DP16K3                 0.0            0.0    0.0   
1             A00005183VODPLUOFIKF8                 0.0          696.0    0.0   
2             A00014061UOTNRBL8FLXR                 6.0            4.0    0.0   
3             A00024081C9YYWHJYQH4L                 0.0           23.0    0.0   
4              A0002574KSGGCRIX1576               195.0            5.0    0.0   

new_groups                       TVOD                             \
offer_group                  PURCHASE                              
device_class  Web Living Room Devices Mobile Devices Others  Web   
0             0.0                 0.0            3.0    0.0  0.0   
1             0.0                 0.0            0.0    0.0  0.0   
2             0.0                 0.0            0.0    0.0  0.0   
3             9.0                 0.0            0.0    0.0  0.0   
4             0.0                 0.0            0.0    0.0  0.0   

new_groups                                                   
offer_group               RENTAL                             
device_class Living Room Devices Mobile Devices Others  Web  
0                            0.0            0.0    0.0  0.0  
1                            0.0            0.0    0.0  0.0  
2                            0.0            0.0    0.0  0.0  
3                            0.0            0.0    0.0  0.0  
4                            0.0            0.0    0.0  0.0

In [332]:
full = pd.merge(left=t,right=r_pivot, left_on="customer_id", right_on="customer_id", how='left')


C:\Users\dfadeeff\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:551: UserWarning: merging between different levels can give an unintended result (3 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [333]:
print(full.shape)
full.head()

(1646810, 17)


,customer_id,"(customer_id, , )","(SVOD, PRIME, Living Room Devices)","(SVOD, PRIME, Mobile Devices)","(SVOD, PRIME, Others)","(SVOD, PRIME, Web)","(TVOD, PURCHASE, Living Room Devices)","(TVOD, PURCHASE, Mobile Devices)","(TVOD, PURCHASE, Others)","(TVOD, PURCHASE, Web)","(TVOD, RENTAL, Living Room Devices)","(TVOD, RENTAL, Mobile Devices)","(TVOD, RENTAL, Others)","(TVOD, RENTAL, Web)",SVOD,TVOD,class
0,A000023039I6L41DP16K3,A000023039I6L41DP16K3,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,TVOD Only
1,A00005183VODPLUOFIKF8,A00005183VODPLUOFIKF8,0.0,696.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,696.0,0.0,SVOD Only
2,A00014061UOTNRBL8FLXR,A00014061UOTNRBL8FLXR,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,SVOD Only
3,A00024081C9YYWHJYQH4L,A00024081C9YYWHJYQH4L,0.0,23.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0,SVOD Only
4,A0002574KSGGCRIX1576,A0002574KSGGCRIX1576,195.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0,0.0,SVOD Only


In [334]:
full.columns

Index([                              'customer_id',
                           ('customer_id', '', ''),
          ('SVOD', 'PRIME', 'Living Room Devices'),
               ('SVOD', 'PRIME', 'Mobile Devices'),
                       ('SVOD', 'PRIME', 'Others'),
                          ('SVOD', 'PRIME', 'Web'),
       ('TVOD', 'PURCHASE', 'Living Room Devices'),
            ('TVOD', 'PURCHASE', 'Mobile Devices'),
                    ('TVOD', 'PURCHASE', 'Others'),
                       ('TVOD', 'PURCHASE', 'Web'),
         ('TVOD', 'RENTAL', 'Living Room Devices'),
              ('TVOD', 'RENTAL', 'Mobile Devices'),
                      ('TVOD', 'RENTAL', 'Others'),
                         ('TVOD', 'RENTAL', 'Web'),
                                            'SVOD',
                                            'TVOD',
                                           'class'],
      dtype='object')

In [335]:
full['TVOD_LRD'] = full[('TVOD', 'PURCHASE', 'Living Room Devices')] + full[('TVOD', 'RENTAL', 'Living Room Devices')]
full['TVOD_Mobile'] = full[ ('TVOD', 'PURCHASE', 'Mobile Devices')] + full[('TVOD', 'RENTAL', 'Mobile Devices')]
full['TVOD_Others'] = full[('TVOD', 'PURCHASE', 'Others')] + full[('TVOD', 'RENTAL', 'Others')]
full['TVOD_Web'] = full[('TVOD', 'PURCHASE', 'Web')] + full[ ('TVOD', 'RENTAL', 'Web')]

In [336]:
full.head()

,customer_id,"(customer_id, , )","(SVOD, PRIME, Living Room Devices)","(SVOD, PRIME, Mobile Devices)","(SVOD, PRIME, Others)","(SVOD, PRIME, Web)","(TVOD, PURCHASE, Living Room Devices)","(TVOD, PURCHASE, Mobile Devices)","(TVOD, PURCHASE, Others)","(TVOD, PURCHASE, Web)",...,"(TVOD, RENTAL, Mobile Devices)","(TVOD, RENTAL, Others)","(TVOD, RENTAL, Web)",SVOD,TVOD,class,TVOD_LRD,TVOD_Mobile,TVOD_Others,TVOD_Web
0,A000023039I6L41DP16K3,A000023039I6L41DP16K3,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,TVOD Only,0.0,3.0,0.0,0.0
1,A00005183VODPLUOFIKF8,A00005183VODPLUOFIKF8,0.0,696.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,696.0,0.0,SVOD Only,0.0,0.0,0.0,0.0
2,A00014061UOTNRBL8FLXR,A00014061UOTNRBL8FLXR,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,10.0,0.0,SVOD Only,0.0,0.0,0.0,0.0
3,A00024081C9YYWHJYQH4L,A00024081C9YYWHJYQH4L,0.0,23.0,0.0,9.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,32.0,0.0,SVOD Only,0.0,0.0,0.0,0.0
4,A0002574KSGGCRIX1576,A0002574KSGGCRIX1576,195.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,200.0,0.0,SVOD Only,0.0,0.0,0.0,0.0


In [337]:
s_df = full[[('SVOD', 'PRIME', 'Living Room Devices'),  ('SVOD', 'PRIME', 'Mobile Devices'), ('SVOD', 'PRIME', 'Others'),
       ('SVOD', 'PRIME', 'Web'), 'customer_id','SVOD','TVOD','class','TVOD_LRD','TVOD_Mobile','TVOD_Others','TVOD_Web']]

In [338]:
s_df.columns = ['SVOD_LRD','SVOD_Mobile','SVOD_Others','SVOD_Web','customer_id','SVOD','TVOD','class','TVOD_LRD','TVOD_Mobile','TVOD_Others','TVOD_Web']
print(s_df.shape, s_df['customer_id'].nunique())

s_df.head()

(1646810, 12) 1646810


,SVOD_LRD,SVOD_Mobile,SVOD_Others,SVOD_Web,customer_id,SVOD,TVOD,class,TVOD_LRD,TVOD_Mobile,TVOD_Others,TVOD_Web
0,0.0,0.0,0.0,0.0,A000023039I6L41DP16K3,0.0,3.0,TVOD Only,0.0,3.0,0.0,0.0
1,0.0,696.0,0.0,0.0,A00005183VODPLUOFIKF8,696.0,0.0,SVOD Only,0.0,0.0,0.0,0.0
2,6.0,4.0,0.0,0.0,A00014061UOTNRBL8FLXR,10.0,0.0,SVOD Only,0.0,0.0,0.0,0.0
3,0.0,23.0,0.0,9.0,A00024081C9YYWHJYQH4L,32.0,0.0,SVOD Only,0.0,0.0,0.0,0.0
4,195.0,5.0,0.0,0.0,A0002574KSGGCRIX1576,200.0,0.0,SVOD Only,0.0,0.0,0.0,0.0


In [339]:
s_df['SVOD_web_mobile'] = s_df['SVOD_Mobile'] + s_df['SVOD_Others'] + s_df['SVOD_Web']
s_df['TVOD_web_mobile'] = s_df['TVOD_Mobile'] + s_df['TVOD_Others'] + s_df['TVOD_Web']
s_df['LRD_total'] = s_df['SVOD_LRD'] + s_df['TVOD_LRD']

final = s_df[['customer_id','SVOD_LRD','SVOD_web_mobile','TVOD_LRD','TVOD_web_mobile','SVOD','TVOD','LRD_total','class']]

C:\Users\dfadeeff\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\dfadeeff\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\dfadeeff\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [341]:
print(final.shape, final['customer_id'].nunique())
final.head()

(1646810, 9) 1646810


,customer_id,SVOD_LRD,SVOD_web_mobile,TVOD_LRD,TVOD_web_mobile,SVOD,TVOD,LRD_total,class
0,A000023039I6L41DP16K3,0.0,0.0,0.0,3.0,0.0,3.0,0.0,TVOD Only
1,A00005183VODPLUOFIKF8,0.0,696.0,0.0,0.0,696.0,0.0,0.0,SVOD Only
2,A00014061UOTNRBL8FLXR,6.0,4.0,0.0,0.0,10.0,0.0,6.0,SVOD Only
3,A00024081C9YYWHJYQH4L,0.0,32.0,0.0,0.0,32.0,0.0,0.0,SVOD Only
4,A0002574KSGGCRIX1576,195.0,5.0,0.0,0.0,200.0,0.0,195.0,SVOD Only


In [342]:
final.is_copy = False
final['LRD_share'] = final['LRD_total'] / (final['SVOD'] + final['TVOD'])

In [343]:
final.head(10)

,customer_id,SVOD_LRD,SVOD_web_mobile,TVOD_LRD,TVOD_web_mobile,SVOD,TVOD,LRD_total,class,LRD_share
0,A000023039I6L41DP16K3,0.0,0.0,0.0,3.0,0.0,3.0,0.0,TVOD Only,0.000000
1,A00005183VODPLUOFIKF8,0.0,696.0,0.0,0.0,696.0,0.0,0.0,SVOD Only,0.000000
2,A00014061UOTNRBL8FLXR,6.0,4.0,0.0,0.0,10.0,0.0,6.0,SVOD Only,0.600000
3,A00024081C9YYWHJYQH4L,0.0,32.0,0.0,0.0,32.0,0.0,0.0,SVOD Only,0.000000
4,A0002574KSGGCRIX1576,195.0,5.0,0.0,0.0,200.0,0.0,195.0,SVOD Only,0.975000
5,A000258623JVX1KJ20BD2,39.0,3.0,8.0,2.0,42.0,10.0,47.0,Hybrid,0.903846
6,A000263437U4UYW47W0EV,0.0,5.0,0.0,0.0,5.0,0.0,0.0,SVOD Only,0.000000
7,A00028262PCB953KRIBH,0.0,2.0,0.0,0.0,2.0,0.0,0.0,SVOD Only,0.000000
8,A00032741ZETFL6SMNKF2,0.0,33.0,0.0,0.0,33.0,0.0,0.0,SVOD Only,0.000000
9,A0004010U0M2J84O06CB,0.0,2.0,0.0,0.0,2.0,0.0,0.0,SVOD Only,0.000000


In [344]:
final['is_LRD_user'] = np.where( final['LRD_share'] >= final['LRD_share'].describe()[5], 1, 0  ) #compare to median
final['is_hybrid'] = np.where( (final['LRD_share'] < final['LRD_share'].describe()[5]) & (final['LRD_share'] >= 0.2), 1,0  )
final['is_mobile_web'] = np.where( (final['is_LRD_user'] == 0) & (final['is_hybrid'] == 0), 1, 0)

In [345]:
final.head()

,customer_id,SVOD_LRD,SVOD_web_mobile,TVOD_LRD,TVOD_web_mobile,SVOD,TVOD,LRD_total,class,LRD_share,is_LRD_user,is_hybrid,is_mobile_web
0,A000023039I6L41DP16K3,0.0,0.0,0.0,3.0,0.0,3.0,0.0,TVOD Only,0.000,0,0,1
1,A00005183VODPLUOFIKF8,0.0,696.0,0.0,0.0,696.0,0.0,0.0,SVOD Only,0.000,0,0,1
2,A00014061UOTNRBL8FLXR,6.0,4.0,0.0,0.0,10.0,0.0,6.0,SVOD Only,0.600,0,1,0
3,A00024081C9YYWHJYQH4L,0.0,32.0,0.0,0.0,32.0,0.0,0.0,SVOD Only,0.000,0,0,1
4,A0002574KSGGCRIX1576,195.0,5.0,0.0,0.0,200.0,0.0,195.0,SVOD Only,0.975,1,0,0


In [346]:
final['final'] = final['is_LRD_user'] + final['is_hybrid'] + final['is_mobile_web']

In [347]:
#Check
final['final'].sum() == final['customer_id'].nunique()

True

In [348]:
f = pd.DataFrame(final.groupby('class')[['is_LRD_user','is_hybrid','is_mobile_web']].sum().reset_index())

In [349]:
f.to_csv("output_UK.csv")